<img src="https://github.com/nicholasmetherall/digital-earth-pacific-macblue-activities/blob/main/attachments/images/DE_Pacific_banner.JPG?raw=true" width="900"/>
Figure 1.1.a. Jupyter environment + Python notebooks

### Digital Earth Pacific Notebook 1 Train Random Forest Machine Learning (ML) Model

<font color='green'>The objective of this notebook is to train a machine learning model that will allow us to classify an area with the land cover classes defined through the training data. </font>

<font color='blue'>Setup libaries</font>

In [2]:
from pystac_client import Client
from dask.distributed import Client as DaskClient
from odc.stac import load, configure_s3_access
import rasterio as rio
import geopandas as gpd
import pandas as pd
import numpy as np
import xarray as xr
import folium
import utils
from utils import load_data
from utils import mask_and_scale
from sklearn.ensemble import RandomForestClassifier
import odc.geo.xr
import rioxarray
import matplotlib.pyplot as plt

<font color='blue'>Define catalogue</font>

In [3]:
catalogue = "https://earth-search.aws.element84.com/v1"
client = Client.open(catalogue)

<font color='blue'>Define your area of interest - copy and paste the bottom left latitude (min_lat) and the bottem left longitude (min_lon) and the top right latitude (max_lat) and the top right longitude (max_lon)

In [4]:
min_lat = -18.1493873
min_lon = 178.4424302
max_lat = -18.1469528 
max_lon = 178.4452552
bbox = [min_lon, min_lat, max_lon, max_lat]

<font color='red'>Define your time of interest - choose a range of a few months in 2024 using the syntax `datetime="YYYY-MM/YYYY-MM"`</font>

In [4]:
datetime=

<font color='red'>Connect to parallel computers for greater computational capacity - note you should only run this line of code once per notebook activity - do not rerun this line</font>

In [5]:
dask_client = DaskClient(n_workers=1, threads_per_worker=16, memory_limit='16GB')

configure_s3_access(cloud_defaults=True, requester_pays=True)

<font color='red'>Define your training data by entering the name of the data and the column of your attribute table you wish to read the data from noting this should be an integer. First enter the full file name including format inside the brackets `("uc_tdata.geojson")` . Next you will have to name the column properly `column="class_id"`</font>

In [24]:
# Define training data
# 1
gdf = gpd.read_file("")

gdf = gdf.to_crs("EPSG:4326")

# 2
gdf.explore(column="", legend=True)


<font color='red'>Count the number of points in your dataset using the function `print(len(variable))` where the variable is the what you stored the geopandas dataset is in</font>

<font color='green'>Discussion question: Where is the training data from?</font>

<font color='green'>Discussion question: move your mouse around each point and pan around the map. What do you think each class or point colour represents? Class 1,2,3? Hint there is a class for buildings, grasslands and trees. So which class number corresponds to each land cover type?</font>

<font color='red'>Search through the catalogue - define the cloud cover threshold you want. Try changing 80 to 25 here: `{"lt": 25}},` </font>

In [8]:
# Search through catalogue for relevant data
items = client.search(
    collections=["sentinel-2-c1-l2a"],
    bbox=bbox,
    datetime=datetime,
    query={"eo:cloud_cover":{"lt": 80}},
).item_collection()

print(f"Found {len(items)} items")


Found 55 items


<font color='blue'>Load your satellite dataset based on your search parameters defined above</font>

In [9]:
# data = utils.load_data
data = load_data(items, bbox)

<font color='blue'>Set scale and masks and consider the dataset informatoin below including the bands used for analysis:</font>

In [25]:
scaled = mask_and_scale(data)
scaled

<font color='blue'>Explore the satellite image dataset you have loaded</font>

In [26]:
scaled.isel(time=0).odc.explore(bands=("red", "green", "blue"), vmin=0, vmax=0.3)


<font color='blue'>Generate a median image dataset</font>

In [12]:
median = scaled.median("time").compute()
median_image = median.assign_coords(band=["red", "green", "blue", "nir", "swir1", "swir2"])

<font color='blue'>Configure your input training labelled GPS points to train your machine learning model</font>

In [14]:
# First transform the training points to the same CRS as the data
training = gdf.to_crs(median.odc.geobox.crs)

# Next get the X and Y values out of the point geometries
training_da = training.assign(x=training.geometry.x, y=training.geometry.y).to_xarray()

# Now we can use the x and y values (lon, lat) to extract values from the median composite
training_values = (
    median.sel(training_da[["x", "y"]], method="nearest").squeeze().compute().to_pandas()
)



<font color='red'>Run this code and look at the table generated including information about the different spectral band values. You wil need to input the column name inside `training["column name"]`</font>

In [30]:
# Join the training data with the extracted values and remove unnecessary columns
training_array = pd.concat([training[""], training_values], axis=1)
training_array = training_array.drop(
    columns=[
        "y",
        "x",
        "spatial_ref",
    ]
)

# Drop rows where there was no data available
training_array = training_array.dropna()

# Preview our resulting training array
training_array.head()

<font color='blue'>Train the machine learning model</font>

In [16]:
# The classes are the first column
classes = np.array(training_array)[:, 0]

# The observation data is everything after the first column
observations = np.array(training_array)[:, 1:]

# Create a model...
classifier = RandomForestClassifier()

# ...and fit it to the data
model = classifier.fit(observations, classes)

<font color='blue'>Prepare the outputs for visualisation</font>

In [17]:
# Convert to a stacked array of observations
stacked_arrays = median.to_array().stack(dims=["y", "x"]).transpose()

# Predict the classes
predicted = model.predict(stacked_arrays)

# Reshape back to the original 2D array
array = predicted.reshape(len(median.y), len(median.x))

# Convert to an xarray again, because it's easier to work with
predicted_da = xr.DataArray(
    array, coords={"y": median.y, "x": median.x}, dims=["y", "x"]
)

In [18]:
print(predicted_da.dtype)  # Check the dtype of your DataArray
predicted_da = predicted_da.astype('float32')  # Convert to float32

float64


<font color='blue'>Visualise the output</font>

In [31]:
# Put it all on a single interactive map
# center = [np.mean([min_lat[0], max_lat[0]]), np.mean([min_lat[1], max_lat[1]])]
# m = folium.Map(location=center, zoom_start=11)

center = [(min_lat + max_lat) / 2, (min_lon + max_lon) / 2]  # Assuming min_lon and max_lon are defined
m = folium.Map(location=center, zoom_start=13)



# RGB for the median
median.odc.to_rgba(bands=("red", "green", "blue"), vmin=0, vmax=0.3).odc.add_to(m, name="Median Composite")


# Categorical for the predicted classes and for the training data
predicted_da.odc.add_to(m, name="Predicted")
gdf.explore(m=m, column="class_id", legend=True, name="Training Data")

# Layer control
folium.LayerControl().add_to(m)

m

<font color='green'>Discussion question: you now have a model / prediction of the three different classes represented by three different classes: trees, grasslands and buildings. Which colour do you think corresponds to each colour?</font>

<font color='blue'>Write the resulting output to a tif file you can download and open in QGIS then you can play with the different colours in the map and make further customisations</font>

In [20]:
predicted_da.odc.write_cog("predicted_randomforest_model_1.tif", overwrite=True)

PosixPath('predicted_randomforest_model_1.tif')

### Congratulations you have completed this activity. Let the instructors know if you have any questions. Otherwise, move onto the next activity 

<font color='green'>The objective of this notebook was to train the machine learning model that will allow us to classify an area with land cover classes defined through the training data. </font>

<font color='blue'>Finished</font>

In [22]:
print("Finished, well done")

Finished, well done
